In [106]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import process
import pickle
from rich.console import Console
from rich.table import Table

In [107]:
# Recuperar el mismo DataFrame
with open('df_guardado.pkl', 'rb') as f:
    df = pickle.load(f)
dfp = df.copy()

In [108]:
# 9 Normalización de los datos equipos
# Lista de nombres correctos (diccionario base)
equipos_correctos = ["nacional", "medellin", "america", "cali", "tolima", "santa fe", "millonarios",
                    "junior", "union magdalena", "pasto", "equidad", "fortaleza", "boyaca chico", "patriotas",
                    "llaneros", "once caldas", "pereira", "envigado", "alianza petrolera", "aguilas doradas", "jaguares",
                    "cortulua", "huila", "quindio"]

def corregir_nombre(nombre, lista_referencia, umbral=85):
    """
    Busca el nombre más parecido en la lista de referencia.
    Si la similitud es mayor al umbral, lo corrige.
    """
    mejor_match, score, _ = process.extractOne(nombre, lista_referencia)
    if score >= umbral:
        return mejor_match
    else:
        return nombre  # si no llega al umbral, lo deja igual

# Aplicamos corrección automática en todas las columnas relevantes
for col in ["equipo_local", "equipo_visitante"]:
    dfp[col] = dfp[col].apply(lambda x: corregir_nombre(x, equipos_correctos))

print("Datos corregidos:\n", dfp)

Datos corregidos:
       torneo  semestre           ronda  fecha_torneo equipo_local  \
0       2021         1         INICIAL             1  millonarios   
1       2021         1         INICIAL             1       junior   
2       2021         1         INICIAL             1        pasto   
3       2021         1         INICIAL             1     nacional   
4       2021         1         INICIAL             1         cali   
...      ...       ...             ...           ...          ...   
2188    2025         2  CUADRANGULARES             1     medellin   
2189    2025         2  CUADRANGULARES             1    fortaleza   
2190    2025         2  CUADRANGULARES             1  bucaramanga   
2191    2025         2  CUADRANGULARES             1     nacional   
2192    2025         2  CUADRANGULARES             1       junior   

     equipo_visitante  goles_local  goles_visitante  resultado  
0            envigado            1                0      LOCAL  
1            medellin 

In [109]:
# Validar duplicados
print("Valores únicos de equipo_local:")
print(dfp["equipo_local"].unique())

# ver cuántas veces aparece cada uno
print("\nFrecuencia de cada equipo_local:")
print(dfp["equipo_local"].value_counts())

Valores únicos de equipo_local:
['millonarios' 'junior' 'pasto' 'nacional' 'cali' 'tolima' 'bucaramanga'
 'alianza petrolera' 'patriotas' 'medellin' 'envigado' 'america'
 'jaguares' 'pereira' 'equidad' 'once caldas' 'boyaca chico' 'santa fe'
 'aguilas doradas' 'quindio' 'huila' 'cortulua' 'union magdalena'
 'fortaleza' 'llaneros']

Frecuencia de cada equipo_local:
equipo_local
millonarios          126
junior               125
tolima               125
nacional             123
medellin             118
santa fe             115
cali                 113
america              112
bucaramanga          108
aguilas doradas      107
pereira              107
pasto                107
once caldas          107
equidad              106
alianza petrolera    104
envigado             101
jaguares              78
boyaca chico          71
union magdalena       60
patriotas             58
fortaleza             42
huila                 30
cortulua              20
llaneros              20
quindio             

In [110]:
# Ajustar el nombre sami por el del cali
dfp["equipo_visitante"] = df["equipo_visitante"].replace("sami", "cali")
dfp["equipo_local"] = df["equipo_local"].replace("sami", "cali")

In [111]:
# Valores únicos de equipos local y visitante - estos deben coincidir
valores_unicos_l = dfp["equipo_local"].unique()
valores_unicos_v = dfp["equipo_visitante"].unique()

# Total de valores únicos
total_unicos_l = dfp["equipo_local"].nunique()
total_unicos_v = dfp["equipo_visitante"].nunique()

print("Valores únicos en la columna equipo_local:")
print(valores_unicos_l)
print(valores_unicos_v)

print(f"\nTotal de valores únicos local: {total_unicos_l}")
print(f"\nTotal de valores únicos_visitante: {total_unicos_v}")

Valores únicos en la columna equipo_local:
['millonarios' 'junior' 'pasto' 'nacional' 'cali' 'tolima' 'bucaramanga'
 'alianza petrolera' 'patriotas' 'medellin' 'envigado' 'america'
 'jaguares' 'pereira' 'equidad' 'once caldas' 'boyaca chico' 'santa fe'
 'aguilas doradas' 'quindio' 'huila' 'cortulua' 'union magdalena'
 'fortaleza' 'llaneros']
['envigado' 'medellin' 'equidad' 'santa fe' 'jaguares' 'once caldas'
 'boyaca chico' 'pereira' 'america' 'patriotas' 'pasto' 'aguilas doradas'
 'bucaramanga' 'cali' 'nacional' 'junior' 'millonarios' 'tolima'
 'alianza petrolera' 'huila' 'quindio' 'union magdalena' 'cortulua'
 'fortaleza' 'llaneros']

Total de valores únicos local: 25

Total de valores únicos_visitante: 25


In [112]:
# Ajustar el tipo de dato correcto de los goles ya que tenia valor double

dfp['goles_local'] = dfp['goles_local'].astype(int)
dfp['goles_visitante'] = dfp['goles_visitante'].astype(int)

In [113]:
# Visualizar dataset organizado con su tipo de datos correspondientes
dfp.head()

,torneo,semestre,ronda,fecha_torneo,equipo_local,equipo_visitante,goles_local,goles_visitante,resultado
0,2021,1,INICIAL,1,millonarios,envigado,1,0,LOCAL
1,2021,1,INICIAL,1,junior,medellin,1,0,LOCAL
2,2021,1,INICIAL,1,pasto,equidad,2,2,EMPATE
3,2021,1,INICIAL,1,nacional,santa fe,2,0,LOCAL
4,2021,1,INICIAL,1,cali,jaguares,1,0,LOCAL


In [114]:
# Eliminar duplicados, no debe existir que un equipo juegue 2 partidos en una misma fecha

# Filtrar solo la fase inicial
dfp_inicial = dfp[dfp["ronda"].str.lower() == "inicial"]

# Agrupar por torneo, semestre, fecha y equipo (tanto local como visitante)
equipos_fecha = pd.concat([
    dfp_inicial[["torneo", "semestre", "fecha_torneo", "equipo_local"]].rename(columns={"equipo_local": "equipo"}),
    dfp_inicial[["torneo", "semestre", "fecha_torneo", "equipo_visitante"]].rename(columns={"equipo_visitante": "equipo"})
])

# Contar cuántas veces aparece cada equipo en la misma fecha
conteo = equipos_fecha.groupby(["torneo", "semestre", "fecha_torneo", "equipo"]).size().reset_index(name="apariciones")

# Filtrar los que aparecen más de una vez
duplicados = conteo[conteo["apariciones"] > 1]

# Mostrar resultados
if duplicados.empty:
    print("No hay equipos repetidos en la misma fecha del mismo torneo y semestre.")
else:
    print("Equipos con más de una aparición en la misma fecha (posible error):")
    print(duplicados.sort_values(["torneo", "semestre", "fecha_torneo"]))


Equipos con más de una aparición en la misma fecha (posible error):
      torneo  semestre  fecha_torneo       equipo  apariciones
148     2021         1             9         cali            2
218     2021         1            13         cali            2
343     2021         2             1         cali            2
402     2021         2             4         cali            2
470     2021         2             7  once caldas            2
660     2021         2            17         cali            2
779     2022         1             3         cali            2
952     2022         1            11       tolima            2
1237    2022         2             6         cali            2
2964    2024         2            13     medellin            2


In [115]:
# Codigo generado para revisar una fecha en particular, ten solo se deben modificar los parametros correspondientes
dfp_filtrado = dfp[
    (dfp["torneo"].astype(str) == "2024") &
    (dfp["semestre"].astype(str) == "2") &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["fecha_torneo"].astype(int) == 13)
]

# Mostrar resultado
print("Registros del torneo 2021, semestre 1, fecha 9:")
print(dfp_filtrado)

Registros del torneo 2021, semestre 1, fecha 9:
      torneo  semestre    ronda  fecha_torneo       equipo_local  \
1651    2024         2  INICIAL            13               cali   
1652    2024         2  INICIAL            13           envigado   
1653    2024         2  INICIAL            13       boyaca chico   
1654    2024         2  INICIAL            13              pasto   
1655    2024         2  INICIAL            13  alianza petrolera   
1656    2024         2  INICIAL            13           medellin   
1657    2024         2  INICIAL            13           santa fe   
1658    2024         2  INICIAL            13             junior   
1659    2024         2  INICIAL            13          fortaleza   
1660    2024         2  INICIAL            13            pereira   

     equipo_visitante  goles_local  goles_visitante  resultado  
1651         medellin            0                1  VISITANTE  
1652      bucaramanga            0                0     EMPATE  
1653    

In [116]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2024) &
    (dfp["semestre"].astype(int) == 2) &
    (dfp["fecha_torneo"].astype(int) == 13) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_visitante"].str.lower() == "medellin")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local", "equipo_visitante"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_visitante"] = "millonarios"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_visitante"]])

Antes del cambio:
      torneo  semestre  fecha_torneo equipo_local equipo_visitante
1651    2024         2            13         cali         medellin

Después del cambio:
      torneo  semestre  fecha_torneo equipo_visitante
1651    2024         2            13      millonarios


In [117]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2022) &
    (dfp["semestre"].astype(int) == 2) &
    (dfp["fecha_torneo"].astype(int) == 6) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "cali")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_local"] = "america"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

Antes del cambio:
     torneo  semestre  fecha_torneo equipo_local
696    2022         2             6         cali

Después del cambio:
     torneo  semestre  fecha_torneo equipo_local
696    2022         2             6      america


In [118]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2022) &
    (dfp["semestre"].astype(int) == 1) &
    (dfp["fecha_torneo"].astype(int) == 11) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "tolima") & 
    (dfp["equipo_visitante"].str.lower() == "junior")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local", "equipo_visitante"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_local"] = "pereira"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])


Antes del cambio:
     torneo  semestre  fecha_torneo equipo_local equipo_visitante
520    2022         1            11       tolima           junior

Después del cambio:
     torneo  semestre  fecha_torneo equipo_local
520    2022         1            11      pereira


In [119]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2021) &
    (dfp["semestre"].astype(int) == 2) &
    (dfp["fecha_torneo"].astype(int) == 17) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "cali")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_local"] = "america"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

Antes del cambio:
     torneo  semestre  fecha_torneo equipo_local
350    2021         2            17         cali

Después del cambio:
     torneo  semestre  fecha_torneo equipo_local
350    2021         2            17      america


In [120]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2021) &
    (dfp["semestre"].astype(int) == 2) &
    (dfp["fecha_torneo"].astype(int) == 7) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "nacional") & 
    (dfp["equipo_visitante"].str.lower() == "once caldas")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local", "equipo_visitante"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_visitante"] = "aguilas doradas"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_visitante"]])

Antes del cambio:
     torneo  semestre  fecha_torneo equipo_local equipo_visitante
253    2021         2             7     nacional      once caldas

Después del cambio:
     torneo  semestre  fecha_torneo equipo_visitante
253    2021         2             7  aguilas doradas


In [121]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2021) &
    (dfp["semestre"].astype(int) == 2) &
    (dfp["fecha_torneo"].astype(int) == 4) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "cali")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_local"] = "america"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

Antes del cambio:
     torneo  semestre  fecha_torneo equipo_local
216    2021         2             4         cali

Después del cambio:
     torneo  semestre  fecha_torneo equipo_local
216    2021         2             4      america


In [122]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2021) &
    (dfp["semestre"].astype(int) == 2) &
    (dfp["fecha_torneo"].astype(int) == 1) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "cali")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_local"] = "america"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

Antes del cambio:
     torneo  semestre  fecha_torneo equipo_local
186    2021         2             1         cali

Después del cambio:
     torneo  semestre  fecha_torneo equipo_local
186    2021         2             1      america


In [123]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2021) &
    (dfp["semestre"].astype(int) == 1) &
    (dfp["fecha_torneo"].astype(int) == 9) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_local"].str.lower() == "cali")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_local"] = "america"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_local"]])

Antes del cambio:
    torneo  semestre  fecha_torneo equipo_local
73    2021         1             9         cali

Después del cambio:
    torneo  semestre  fecha_torneo equipo_local
73    2021         1             9      america


In [124]:
# Filtro para identificar el registro que deseas cambiar
condicion = (
    (dfp["torneo"].astype(int) == 2021) &
    (dfp["semestre"].astype(int) == 1) &
    (dfp["fecha_torneo"].astype(int) == 13) &
    (dfp["ronda"].str.lower() == "inicial") &
    (dfp["equipo_visitante"].str.lower() == "cali")
)

# Verificar antes del cambio
print("Antes del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_visitante"]])

# Realizar el cambio
dfp.loc[condicion, "equipo_visitante"] = "america"

# Confirmar el cambio
print("\nDespués del cambio:")
print(dfp.loc[condicion, ["torneo", "semestre", "fecha_torneo", "equipo_visitante"]])

Antes del cambio:
     torneo  semestre  fecha_torneo equipo_visitante
111    2021         1            13             cali

Después del cambio:
     torneo  semestre  fecha_torneo equipo_visitante
111    2021         1            13          america


In [125]:
# Cantidad de registros
cantidad_registros = len(dfp)
cantidad_variables = dfp.shape[1]

# Obtener primer y último registro
primer_registro = dfp.iloc[0].to_dict()
ultimo_registro = dfp.iloc[-1].to_dict()

# Crear consola
console = Console()

# Crear tabla principal
tabla = Table(title="RESUMEN DATA SET LIGA COLMBIANA", style="bold cyan")

tabla.add_column("Descripción", style="bold yellow", justify="center")
tabla.add_column("Valor", style="bold black", justify="left")

# Agregar cantidad de registros
tabla.add_row("Cantidad de registros", str(cantidad_registros))
tabla.add_row("Cantidad de columnas", str(cantidad_variables))

# Agregar primer registro
tabla.add_row("Primer registro", str(primer_registro))

# Agregar último registro
tabla.add_row("Último registro", str(ultimo_registro))

# Mostrar tabla en consola
console.print(tabla)

                                          RESUMEN DATA SET LIGA COLMBIANA                                          
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Descripción      ┃ Valor                                                                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Cantidad de registros │ 2193                                                                                    │
│ Cantidad de columnas  │ 9                                                                                       │
│    Primer registro    │ {'torneo': 2021, 'semestre': 1, 'ronda': 'INICIAL', 'fecha_torneo': 1, 'equipo_local':  │
│                       │ 'millonarios', 'equipo_visitante': 'envigado', 'goles_local': 1, 'goles_visitante': 0,  │
│                       │ 'resultado': 'LOCAL'}                                                                   │
│    Último registro    │ {'torneo': 2025, 'semestre': 2, 'ronda': 'CUADRANGULARES', 'fecha_torneo': 1,           │
│                       │ 'equipo_local': 'junior', 'equipo_visitante': 'america', 'goles_local': 2,              │
│                       │ 'goles_visitante': 1, 'resultado': 'LOCAL'}                                             │
└───────────────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

In [126]:
with open('dfp_guardado.pkl', 'wb') as p:
    pickle.dump(dfp, p)